In [ ]:
# %% [markdown]
# Проверка RAG-бота локально
# В этом ноутбуке вы сможете:
# 1. Загрузить чанки из data/responses.json
# 2. Создать FAISS-индекс
# 3. Настроить RetrievalQA и протестировать ответы

# %%
# Установим переменные окружения (замените на ваши данные или используйте .env)
import os

# %%
# Импортируем необходимые библиотеки
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI       # <<-- заменили Import
from langchain.chains import RetrievalQA
import json
from pathlib import Path


In [20]:
# 1. Эмбеддинги (должны совпадать с теми, что использовались при создании индекса)
embeddings = OpenAIEmbeddings(
    model='text-embedding-ada-002',
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    openai_api_base=os.getenv('OPENAI_API_BASE')
)

chat = ChatOpenAI(
    model_name='gpt-4o-mini',
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    openai_api_base=os.getenv('OPENAI_API_BASE'),
    temperature=0
)

# 2. Загрузи FAISS-индекс
vectorstore = FAISS.load_local(
    'faiss_index',  # путь к папке с индексом
    embeddings,
    allow_dangerous_deserialization=True
)

retriever = vectorstore.as_retriever(search_kwargs={'k': 7})
qa = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

In [ ]:
# 1. Ваш вопрос
question = "Когда проходит устный экзамен?"

# 2. Получаем релевантные документы
docs = retriever.get_relevant_documents(question)

# 3. Формируем контекст как в rag_bot.py (5 FAQ + 2 rules)
faq_docs = [d for d in docs if d.metadata.get("source") == "faq"]
rules_docs = [d for d in docs if d.metadata.get("source") == "rules"]
selected = faq_docs[:5] + rules_docs[:2]

context_parts = []
for d in selected:
    if d.metadata.get("source") == "rules":
        section = d.metadata.get("section")
        if section:
            context_parts.append(f"[Правила, пункт {section}]: {d.page_content}")
        else:
            context_parts.append(f"[Правила]: {d.page_content}")
    else:
        context_parts.append(f"[FAQ]: {d.page_content}")

context = "\n".join(context_parts)

print("=== КОНТЕКСТ ===")
print(context)
print("\n=== PROMPT ===")
prompt = prompt_template.format(text=context, query=question)
print(prompt)

# 4. Получаем финальный ответ
result = chat.invoke(prompt)
print("\n=== ОТВЕТ МОДЕЛИ ===")
print(result.content.strip())

In [12]:
# %% [markdown]
# 1) Загрузка и разбиение чанков

# %% 
input_path = Path('data/responses.json')
with open(input_path, encoding='utf-8') as f:
    chunks = json.load(f)

splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts, metadatas = [], []
for e in chunks:
    for part in splitter.split_text(e['text']):
        texts.append(part)
        metadatas.append(e['metadata'])


In [13]:
# %% [markdown]
# 2) Эмбеддинги

# %% 
embeddings = OpenAIEmbeddings(
    model='text-embedding-ada-002',
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    openai_api_base=os.getenv('OPENAI_API_BASE')
)


In [14]:
# %% [markdown]
# 4) RetrievalQA с ChatOpenAI

# %% 
chat = ChatOpenAI(
    model_name='gpt-4o-mini',
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    openai_api_base=os.getenv('OPENAI_API_BASE'),
    temperature=0
)

# При загрузке индекса разрешаем десериализацию
vectorstore = FAISS.load_local(
    'faiss_index',
    embeddings,
    allow_dangerous_deserialization=True
)

retriever = vectorstore.as_retriever(search_kwargs={'k': 5})
qa = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)


C:\Users\Artem Khakimov\AppData\Local\Temp\ipykernel_3272\1689844017.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [ ]:
# %% [markdown]
# 5) Тест запросов

# %% 
questions = [
    "Когда проходит устный экзамен?",
    "Как подать заявление на бюджет?",
    "Куда писать для апелляции?",
    "Что делать, если я передумал выбирать кафедру после отбора?",
    "Как и где я могу посмотреть свои результаты контеста и собеседования?",
    "Нужен ли оригинал диплома бакалавра для подачи документов?",
    "Можно ли изменить приоритеты выбора кафедр после отправки заявления?",
    "Когда объявят список зачисленных и где его искать?",
    "Какие документы прикладывать в разделе «Индивидуальные достижения»?",
    "Как убедиться, что моё заявление действительно дошло до приёмной комиссии?",
    "Что делать, если не пришло письмо с данными для Яндекс.Контеста?",
    "Есть ли дедлайн для добавления новых ИД и как это сделать?",
    "Куда можно обратиться, если возникли технические проблемы с порталом?",
    "Можно ли подать документы на контракт и позже сменить на бюджет?",
    "Каковы основные этапы поступления и на что обратить внимание в первую очередь?"
]


for q in questions:
    print("Q:", q)
    # вместо qa.run используем __call__
    result = qa({"query": q})
    print("A:", result["result"])
    print("-" * 40)


Q: Когда проходит устный экзамен?
A: Баллы за устный экзамен появляются в течение 1–2 рабочих дней после его проведения. Обычно результаты передают в понедельник, а публикуют во вторник. Однако конкретная дата устного экзамена не указана в предоставленной информации.
----------------------------------------
Q: Как подать заявление на бюджет?
A: Подача заявления на бюджет начинается с 20 июня. Заявление можно подать только через Госуслуги, кроме иностранных граждан по квоте. Рекомендуется подавать заявление как можно раньше, так как в последние дни могут возникнуть ошибки, которые могут привести к отказу. Также необходимо подать согласие на зачисление, которое можно сделать либо через Госуслуги, либо в бумажном виде в приёмную комиссию.
----------------------------------------
Q: Куда писать для апелляции?
A: Для подачи апелляции отправьте письмо на pk.magistr@mipt.ru.
----------------------------------------
Q: Что делать, если я передумал выбирать кафедру после отбора?
A: Я не знаю.
-

In [17]:
# %% [markdown]
# 5) Тест запросов

# %% 
questions = [
    "Здравствуйте, помогите пожалуйста разобраться со следующими вопросами:\n1. Где-то можно посмотреть примеры заданий по математике? \n2. В какой момент и где указываются кафедры, на которые я бы хотела поступать?\n3. В рассылке о вступительных испытаниях было написано, что нужно 2ым приорететом указать 1.01.04.02 Прикладная математика и информатика, а я этого не сделала. Если я отредактирую заявление на гос услугах, меня успеют обработать ко вступительным, которые будут 22-23?\n4. Обязательно ли согласие на зачисление для прохождения вступительных испытаний? Я подала документы в еще одну физтех-школу."
]


for q in questions:
    print("Q:", q)
    # вместо qa.run используем __call__
    result = qa({"query": q})
    print("A:", result["result"])
    print("-" * 40)


Q: Здравствуйте, помогите пожалуйста разобраться со следующими вопросами:
1. Где-то можно посмотреть примеры заданий по математике? 
2. В какой момент и где указываются кафедры, на которые я бы хотела поступать?
3. В рассылке о вступительных испытаниях было написано, что нужно 2ым приорететом указать 1.01.04.02 Прикладная математика и информатика, а я этого не сделала. Если я отредактирую заявление на гос услугах, меня успеют обработать ко вступительным, которые будут 22-23?
4. Обязательно ли согласие на зачисление для прохождения вступительных испытаний? Я подала документы в еще одну физтех-школу.
A: 1. Примеры заданий по математике и материалы прошлых лет доступны на сайте приёмной комиссии (https://mipt.ru/admission/exams) и в разделе «Материалы» на портале exams.mipt.ru.

2. Кафедры, на которые вы хотите поступать, указываются в заявлении при подаче документов. Ссылка для изменения приоритетов кафедр приходит на почту, если вы хотите внести изменения.

3. Если вы отредактируете зая